 ## Import needed libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('anomaly.csv')
df.head()

,Date,Power,Detector,Quality
0,1-Jan-16,96,8,Good
1,2-Jan-16,96,10,Good
2,3-Jan-16,91,8,Good
3,4-Jan-16,97,9,Good
4,5-Jan-16,91,11,Good


In [3]:
print(df.groupby('Quality')['Quality'].count())

df.drop(['Date'], axis=1, inplace=True)
print(df.head())

#If there are missing entries, drop them.
df.dropna(inplace=True,axis=1)

Quality
Bad      407
Good    1054
Name: Quality, dtype: int64
   Power  Detector Quality
0     96         8    Good
1     96        10    Good
2     91         8    Good
3     97         9    Good
4     91        11    Good


In [4]:
#COnvert non-numeric to numeric
df.Quality[df.Quality == 'Good'] = 1
df.Quality[df.Quality == 'Bad'] = 2


good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for good data points

<ipython-input-4-fcd048b652f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Good'] = 1
<ipython-input-4-fcd048b652f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality == 'Bad'] = 2


In [5]:
# good_mask = df['Quality'] == 'Good' #All good to be True for good data points
# bad_mask = df['Quality'] == 'Bad' #All values False for good data points
# bad_mask

In [6]:
# good_mask

In [7]:
df.drop('Quality',axis=1,inplace=True)

df_good = df[good_mask]
df_bad = df[bad_mask]

print(f"Good count: {len(df_good)}")
print(f"Bad count: {len(df_bad)}")

Good count: 1054
Bad count: 407


In [8]:
# df.head()

In [9]:
# This is the feature vector that goes to the neural net
x_good = df_good.values
x_bad = df_bad.values

from sklearn.model_selection import train_test_split

x_good_train, x_good_test = train_test_split(
        x_good, test_size=0.25, random_state=42)

print(f"Good train count: {len(x_good_train)}")
print(f"Good test count: {len(x_good_test)}")

Good train count: 790
Good test count: 264


In [10]:
#Define the autoencoder model
#Since we're dealing with numeric values we can use only Dense layers.

from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(x_good.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 33        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 22        
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_good_train,x_good_train,verbose=1,epochs=150)


Epoch 1/150
25/25 [==============================] - 1s 1ms/step - loss: 2080.8624
Epoch 2/150
25/25 [==============================] - 0s 1ms/step - loss: 1458.4153
Epoch 3/150
25/25 [==============================] - 0s 1ms/step - loss: 884.9508
Epoch 4/150
25/25 [==============================] - 0s 1ms/step - loss: 434.0728
Epoch 5/150
25/25 [==============================] - 0s 1ms/step - loss: 151.9840
Epoch 6/150
25/25 [==============================] - 0s 1ms/step - loss: 33.7151
Epoch 7/150
25/25 [==============================] - 0s 1ms/step - loss: 4.5807
Epoch 8/150
25/25 [==============================] - 0s 1ms/step - loss: 1.4233
Epoch 9/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2705
Epoch 10/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2156
Epoch 11/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2626
Epoch 12/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2837
Epoch 13/150
25/25 [====

25/25 [==============================] - 0s 917us/step - loss: 1.2601
Epoch 101/150
25/25 [==============================] - 0s 916us/step - loss: 1.2771
Epoch 102/150
25/25 [==============================] - 0s 917us/step - loss: 1.1776
Epoch 103/150
25/25 [==============================] - 0s 958us/step - loss: 1.1863
Epoch 104/150
25/25 [==============================] - 0s 875us/step - loss: 1.2230
Epoch 105/150
25/25 [==============================] - 0s 958us/step - loss: 1.2226
Epoch 106/150
25/25 [==============================] - 0s 917us/step - loss: 1.2153
Epoch 107/150
25/25 [==============================] - 0s 958us/step - loss: 1.2771
Epoch 108/150
25/25 [==============================] - 0s 1ms/step - loss: 1.1869
Epoch 109/150
25/25 [==============================] - 0s 875us/step - loss: 1.1737
Epoch 110/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2254
Epoch 111/150
25/25 [==============================] - 0s 875us/step - loss: 1.2322
Epoch 112/

In [12]:
pred = model.predict(x_good_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))

pred = model.predict(x_good)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good))

pred = model.predict(x_bad)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

print(f"Insample Good Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Good Score (RMSE): {score2}")
print(f"Bad sample Score (RMSE): {score3}")

Insample Good Score (RMSE): 1.0785946323431017
Out of Sample Good Score (RMSE): 1.0770836137754523
Bad sample Score (RMSE): 2.6979655123631243
